In [1]:
from langchain_openai import OpenAI

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
llm = OpenAI(temperature=0.4)

In [27]:
text = "What is the capital of india."

In [28]:
llm.predict(text)

'\n\nThe capital of India is New Delhi.'

In [6]:
from langchain import HuggingFaceHub

In [7]:
hugginface_api_token = os.environ["HUGGINGFACE_API_KEY"]

In [35]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature": 0.2 , "max_length":64}, huggingfacehub_api_token=hugginface_api_token)

U:\LangChainProjects\langEnv\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [36]:
output = llm_huggingface.predict("Full form of DC ?")
output

ConnectionError: (ProtocolError('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None)), '(Request ID: d4527e92-3859-4044-9d3d-66dc74f27207)')

In [10]:
llm_huggingface.predict("Write a tag line for superman.")

'he is a sailor'

In [11]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(input_variables = ["country"], template = "What is the capital of {country}")

template.format(country = "India")

'What is the capital of India'

In [14]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = template)
chain.invoke("India")["text"]

'\n\nThe capital of India is New Delhi.'

## Combining Multiple Chains Using Simple sequential Chain

In [42]:
template = "What is the capital of {country}"
capital_template = PromptTemplate(input_variables=['country'], template = template)

capital_chain = LLMChain(llm = llm, prompt=capital_template)

famous_places_template = PromptTemplate(input_variables=["capital"], template="Suggest some places to viist in {capital}")

In [16]:
famous_chain = LLMChain(llm = llm, prompt=famous_places_template)

In [19]:
from langchain.chains import SimpleSequentialChain

main_chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])

main_chain.invoke("india")

{'input': 'india',
 'output': " Some must-visit places in New Delhi include:\n\n1. Red Fort: This iconic monument is a UNESCO World Heritage Site and was the main residence of the Mughal emperors for nearly 200 years.\n\n2. India Gate: A war memorial dedicated to the soldiers who lost their lives in World War I, this is a popular spot for picnics and evening strolls.\n\n3. Qutub Minar: Another UNESCO World Heritage Site, this 73-meter tall tower is the tallest brick minaret in the world and is surrounded by beautiful gardens.\n\n4. Lotus Temple: This stunning temple, shaped like a lotus flower, is a Bahá'í House of Worship and is open to people of all religions.\n\n5. Humayun's Tomb: This grand mausoleum is the final resting place of Mughal Emperor Humayun and is a beautiful example of Mughal architecture.\n\n6. Jama Masjid: One of the largest mosques in India, this impressive structure can hold up to 25,000 worshippers at a time.\n\n7. Chandni Chowk: This bustling market in Old Delhi 

## Sequential Chain

In [45]:
capital_chain = LLMChain(llm = llm, prompt=capital_template, output_key="capital")
famous_chain = LLMChain(llm = llm, prompt=famous_places_template, output_key = "places")

In [48]:
capital_chain.invoke("India")

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.'}

In [39]:
from langchain.chains import SequentialChain

In [49]:
chain = SequentialChain(chains = [capital_chain, famous_chain], input_variables=["country"], output_variables=["capital", "places"])

In [51]:
chain.invoke({"country":"India"})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some popular places to visit in New Delhi are:\n\n1. Red Fort - a historic fort built in the 17th century by Mughal emperor Shah Jahan\n2. India Gate - a war memorial and popular tourist spot\n3. Qutub Minar - a UNESCO World Heritage Site and the tallest brick minaret in the world\n4. Lotus Temple - a Bahá'í House of Worship known for its beautiful lotus-shaped architecture\n5. Humayun's Tomb - a stunning Mughal-era mausoleum and UNESCO World Heritage Site\n6. Jama Masjid - one of the largest mosques in India and a popular religious site\n7. Akshardham Temple - a modern Hindu temple known for its intricate architecture and spiritual atmosphere\n8. Chandni Chowk - a bustling market in Old Delhi known for its street food, shopping, and cultural experience\n9. Rashtrapati Bhavan - the official residence of the President of India, with beautiful gardens and architecture\n10. National Gallery of Modern A

## ChatModels And Prompt Template + LLM + Output Parser

In [57]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser, HumanMessage, SystemMessage, AIMessage

In [58]:
chatllm = ChatOpenAI(temperature=0.6)

In [61]:
chatllm.invoke([
    SystemMessage(content="You are STEM field expert and you need to help to solve the queries."),
    HumanMessage(content="What is Gravity according to Einstein?")
]).content

"According to Einstein's theory of general relativity, gravity is not seen as a force, but rather as a curvature of spacetime caused by the presence of mass and energy. In this theory, objects with mass and energy cause spacetime to curve, and other objects move along the curvature of spacetime, giving the appearance of being attracted to each other. This curvature is what we perceive as gravity. In simpler terms, gravity is the result of the warping of spacetime by mass and energy."

In [53]:
class CommaSeperateOutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [54]:
template = "You are a helpful assistant. When the use given any input, you should generate 5 words synonyms in a comma seperated format."
human_template = "{text}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [62]:
chain = chat_prompt|chatllm|CommaSeperateOutput()

In [64]:
chain.invoke({"text":"handsome"})

['attractive', ' good-looking', ' charming', ' dashing', ' appealing']